In [1]:
import torch
import json_repair
import pandas as pd

from tqdm import tqdm
from datasets import Dataset

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/unsloth/__init__.py:44: UserWarning: Unsloth: 'CUDA_VISIBLE_DEVICES' is currently 0,1,2 
Unsloth currently does not support multi GPU setups - but we are working on it!
Multiple CUDA devices detected but we require a single device.
We will override CUDA_VISIBLE_DEVICES to first device: 0.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [64]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name="unsloth/Llama-3.3-70B-Instruct-bnb-4bit",
    model_name="unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.12.4: Fast Qwen2 patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA H100 NVL. Max memory: 93.003 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [65]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

Unsloth 2024.12.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## data

In [66]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/dataset_files/comics_dataset.csv")

In [67]:
def get_emotions(row):

    utterance_emotions = row.emotion
    utterance_emotions_l = []
    emotion_class_labels = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy"]

    if utterance_emotions == 'Neutral':
        
        utterance_emotions_l.append(utterance_emotions)
    
    else:
        utterance_emotions = utterance_emotions.split("-")
       
        #emotion_annotation_l = []

        for idx, emotion_annotation in enumerate(utterance_emotions):

            if '0' not in emotion_annotation:
         
                #emotion_annotation_l.append(emotion_class_labels[idx])
                #utterance_emotions_l.append(emotion_annotation[:-1])
                utterance_emotions_l.append(emotion_class_labels[idx])
            
        #title_emotions_l.append(emotion_annotation_l)
                

    return utterance_emotions_l

In [68]:
df['emotion_u'] = df.apply(lambda row: get_emotions(row), axis=1)

In [76]:
df_train = df[df.split == "TRAIN"].reset_index(drop=True)

In [77]:
len(df_train)

4596

### Comics dataset

In [78]:
# def build_instruction():
#     emotion_classes = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy", "Neutral"]
#     formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
#     instruction = f"""### Emotion Analysis Expert Role

# You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.

# INPUT:
# - You will receive a single utterance from a comic book
# - The utterance may express one or multiple emotions

# TASK:
# 1. Carefully analyze the emotional context and tone of the utterance
# 2. Identify applicable emotions from the following classes:
#    {formatted_classes}

# OUTPUT REQUIREMENTS:
# - Format: JSON object with a single key "list_emotion_classes"
# - Value: Array of one or more emotion classes as strings

# IMPORTANT NOTES:
# - Do not include any explanations in the output, only the JSON object

# """
#     return instruction

In [79]:
def build_instruction():
    emotion_classes = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy", "Neutral"]
    formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
    instruction = f"""### Emotion Analysis Expert Role

You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content while considering conversational context.

INPUT:
- You will receive a list of 6 consecutive utterances from a comic book:
  * The first 5 utterances provide conversational context
  * The 6th (last) utterance is the one to be classified
- Each utterance may express one or multiple emotions

TASK:
1. Read through the context utterances to understand the emotional flow
2. Carefully analyze the emotional context, tone, and potential emotional shifts in the final utterance
3. Identify applicable emotions for the final utterance only from the following classes:
   {formatted_classes}

OUTPUT REQUIREMENTS:
- Format: JSON object with a single key "list_emotion_classes"
- Value: Array of one or more emotion classes as strings
- The classification should be for the final utterance only, using previous utterances as context

IMPORTANT NOTES:
- Do not include any explanations in the output, only the JSON object
- Use the context to better understand emotional transitions and current emotional state
- Context utterances may reveal emotional buildup or shifts that influence the final utterance

"""
    return instruction

In [80]:
instruction = build_instruction()

In [81]:
human_msg_l = []
assistant_msg_l = []

for _, row in df_train.iterrows():
        
        prompt = instruction.replace("<comic_title>", row['comics_title']).replace("<speaker_id>", row['speaker_id']).replace("<utterance>", row['utterance'])
        
        human_msg = {'from': 'human', 'value': prompt}

        #obj = {"list_emotion_classes": row['emotion_u']}
        obj = row['emotion_u']
        #assistant_msg = {'from': 'gpt', 'value': obj}
        assistant_msg = {'from': 'gpt', 'value': f'{{"list_emotion_classes": {obj}}}'}


        human_msg_l.append(human_msg)
        assistant_msg_l.append(assistant_msg)
        

In [82]:
comics_dataset = []

for i in range(len(human_msg_l)):
    
    #obj = {"list_emotion_classes": ["Anger", "Fear"]}

    comics_dataset.append([human_msg_l[i], assistant_msg_l[i]])

In [83]:
len(comics_dataset)

4596

In [84]:
def fix_comics_dataset(comics_dataset):
    fixed_comics_dataset = []
    for conversation in comics_dataset:
        fixed_conversation = []
        for message in conversation:
            if isinstance(message['value'], list):  # If the 'value' is a list of emotions
                message['value'] = ', '.join(message['value'])  # Join the list into a string
            fixed_conversation.append(message)
        fixed_comics_dataset.append(fixed_conversation)
    return fixed_comics_dataset

In [85]:
fixed_comics_dataset = fix_comics_dataset(comics_dataset)

In [86]:
dataset = Dataset.from_dict({
    'conversations': fixed_comics_dataset
})

In [87]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 4596
})

In [88]:
dataset[0]['conversations']

[{'from': 'human',
  'value': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Analyze the following comic book utterance and identify the emotions expressed.\n\nGiven the following information:\n- Comic Title: Nightwing\n- Speaker ID: NIGHTWING\n- Utterance: CAN YOU WALK?\n\nYour task is to:\n1. Carefully analyze the emotional context and tone of the utterance\n2. Consider the comic\'s context and speaker\'s identity\n3. Identify applicable emotions from these classes:\n   "Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy", "Neutral"\n\n### Response:\nProvide your response in the following JSON format:\n{\n    "list_emotion_classes": ["emotion1", ... ]\n}\n\n### Note:\n- Return only the JSON object with a single key "list_emotion_classes" and value is an array of one or more emotion classes as strings\n- In

In [89]:
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template_comics(examples):
    messages = examples["conversations"]
    #messages = examples['input'] + examples['output']
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


In [90]:
comics_dataset = dataset.map(apply_template_comics, batched=True)

Map:   0%|          | 0/4596 [00:00<?, ? examples/s]

In [91]:
comics_dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 4596
})

In [92]:
def split_dataset(dataset, train_ratio=0.8):
    train_test = dataset.train_test_split(test_size=1 - train_ratio)
    return train_test

dataset_split = split_dataset(comics_dataset)

In [93]:
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

In [94]:
train_dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 3676
})

In [96]:
print(train_dataset[1360]['text'])

<|im_start|>user
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Analyze the following comic book utterance and identify the emotions expressed.

Given the following information:
- Comic Title: American Vampire
- Speaker ID: Skinner
- Utterance: I GET IT, BUT STILL, WHAT IF I HAVE ANOTHER ONSET OF WHATEVER THE HELL THAT WAS?

Your task is to:
1. Carefully analyze the emotional context and tone of the utterance
2. Consider the comic's context and speaker's identity
3. Identify applicable emotions from these classes:
   "Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy", "Neutral"

### Response:
Provide your response in the following JSON format:
{
    "list_emotion_classes": ["emotion1", ... ]
}

### Note:
- Return only the JSON object with a single key "list_emotion_classes" and value is an array of one or more

In [99]:
args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="cosine",
        per_device_train_batch_size=16,
        gradient_accumulation_steps=2,
        num_train_epochs=5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=100,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        
        eval_strategy="steps",  # Run evaluation during training (can also use "epoch")
        eval_steps=25,  # Perform evaluation every 50 steps
        save_strategy="steps",  # Save the model every few steps
        save_steps=50,  # Save every 200 steps
        load_best_model_at_end=True,
    
        output_dir="ft_uns_comics_qwen2.5",
        seed=0,
    )

In [100]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,  # Replace with your train dataset
    eval_dataset=eval_dataset, 
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=args,
)

Map (num_proc=2):   0%|          | 0/3676 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/920 [00:00<?, ? examples/s]

In [101]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,676 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 2
\        /    Total batch size = 32 | Total steps = 575
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss,Validation Loss
25,No log,0.228452
50,No log,0.200484
75,No log,0.196264
100,0.739900,0.195323
125,0.739900,0.193926
150,0.739900,0.193246
175,0.739900,0.193512
200,0.336700,0.190043
225,0.336700,0.189271
250,0.336700,0.204663


Could not locate the best model at ft_uns_comics_qwen2.5/checkpoint-225/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=575, training_loss=0.3092653639420219, metrics={'train_runtime': 1548.7029, 'train_samples_per_second': 11.868, 'train_steps_per_second': 0.371, 'total_flos': 2.30652049324032e+17, 'train_loss': 0.3092653639420219, 'epoch': 5.0})

In [102]:
model = FastLanguageModel.for_inference(model)

In [103]:
df_test = df[df.split == "TEST"].reset_index(drop=True)

In [104]:
human_msg_l = []
assistant_msg_l = []

for _, row in df_test.iterrows():
        
        prompt = instruction.replace("<comic_title>", row['comics_title']).replace("<speaker_id>", row['speaker_id']).replace("<utterance>", row['utterance'])
        
        human_msg = {'role': 'user', 'content': prompt}
        
        #obj = {"list_emotion_classes": row['emotion_u']}
        obj = row['emotion_u']
        assistant_msg = {'role': 'assistant', 'content': ""}
        
        human_msg_l.append(human_msg)
        assistant_msg_l.append(assistant_msg)
        
        

In [105]:
test_messages = []

for i in range(len(human_msg_l)):
    
    #obj = {"list_emotion_classes": ["Anger", "Fear"]}

    test_messages.append([human_msg_l[i], assistant_msg_l[i]])

In [106]:
len(test_messages)

1101

In [107]:
# test_messages = test_messages[:100]

In [108]:
raw_outputs = []

for message in tqdm(test_messages):
    
    inputs = tokenizer.apply_chat_template(
    message,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")
    #print(inputs)
    #break
    
    output = model.generate(input_ids=inputs, max_new_tokens=128)[0]
    
    input_length = inputs.shape[1]
    generated_tokens = output[input_length:]
    
    decoded_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)  
    #decoded_output = tokenizer.decode(output, skip_special_tokens=True)
    raw_outputs.append(decoded_output)
    #break

100%|██████████| 1101/1101 [09:18<00:00,  1.97it/s]


In [109]:
#print(tokenizer.decode(inputs[0]))

In [110]:
len(raw_outputs)

1101

In [111]:
raw_outputs

['{"list_emotion_classes": [\'Fear\', \'Surprise\']}',
 '{"list_emotion_classes": [\'Fear\', \'Surprise\']}',
 '{"list_emotion_classes": [\'Fear\', \'Sadness\']}',
 '{"list_emotion_classes": [\'Joy\']}',
 '{"list_emotion_classes": [\'Joy\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Fear\', \'Surprise\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Fear\', \'Surprise\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Fear\']}',
 '{"list_emotion_classes": [\'Sadness\']}',
 '{"list_emotion_classes": [\'Sadness\']}',
 '{"list_emotion_classes": [\'Sadness\']}',
 '{"list_emotion_classes": [\'Anger\', \'Fear\']}',
 '{"list_emotion_classes": [\'Sadness\', \'Surprise\']}',
 '{"list_emotion_classes": [\'Surprise\']}',
 '{"list_emotion_classes": [\'Joy\']}',
 '{"list_emoti

In [112]:
grounds = df_test.emotion_u.tolist()

In [113]:
len(grounds)

1101

In [114]:
predictions = [e.split('\n\n')[0] for e in raw_outputs]
#predictions = [json.loads(e) for e in raw_outputs]

In [115]:
len(predictions)

1101

In [116]:
predictions

['{"list_emotion_classes": [\'Fear\', \'Surprise\']}',
 '{"list_emotion_classes": [\'Fear\', \'Surprise\']}',
 '{"list_emotion_classes": [\'Fear\', \'Sadness\']}',
 '{"list_emotion_classes": [\'Joy\']}',
 '{"list_emotion_classes": [\'Joy\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Fear\', \'Surprise\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Fear\', \'Surprise\']}',
 '{"list_emotion_classes": [\'Neutral\']}',
 '{"list_emotion_classes": [\'Fear\']}',
 '{"list_emotion_classes": [\'Sadness\']}',
 '{"list_emotion_classes": [\'Sadness\']}',
 '{"list_emotion_classes": [\'Sadness\']}',
 '{"list_emotion_classes": [\'Anger\', \'Fear\']}',
 '{"list_emotion_classes": [\'Sadness\', \'Surprise\']}',
 '{"list_emotion_classes": [\'Surprise\']}',
 '{"list_emotion_classes": [\'Joy\']}',
 '{"list_emoti

In [117]:
predictions = [json_repair.loads(e) for e in predictions]

In [118]:
predictions

[{'list_emotion_classes': ['Fear', 'Surprise']},
 {'list_emotion_classes': ['Fear', 'Surprise']},
 {'list_emotion_classes': ['Fear', 'Sadness']},
 {'list_emotion_classes': ['Joy']},
 {'list_emotion_classes': ['Joy']},
 {'list_emotion_classes': ['Neutral']},
 {'list_emotion_classes': ['Fear', 'Surprise']},
 {'list_emotion_classes': ['Neutral']},
 {'list_emotion_classes': ['Neutral']},
 {'list_emotion_classes': ['Neutral']},
 {'list_emotion_classes': ['Neutral']},
 {'list_emotion_classes': ['Fear', 'Surprise']},
 {'list_emotion_classes': ['Neutral']},
 {'list_emotion_classes': ['Fear']},
 {'list_emotion_classes': ['Sadness']},
 {'list_emotion_classes': ['Sadness']},
 {'list_emotion_classes': ['Sadness']},
 {'list_emotion_classes': ['Anger', 'Fear']},
 {'list_emotion_classes': ['Sadness', 'Surprise']},
 {'list_emotion_classes': ['Surprise']},
 {'list_emotion_classes': ['Joy']},
 {'list_emotion_classes': ['Sadness']},
 {'list_emotion_classes': ['Anger', 'Fear']},
 {'list_emotion_classes': 

In [119]:
preds_l = []
bad_idx = []

for i, pred in enumerate(predictions):
    try:
        
        preds_l.append(pred['list_emotion_classes'])
    except:
        print(i)
        bad_idx.append(i)

In [120]:
grounds = [item for i, item in enumerate(grounds) if i not in bad_idx]

In [121]:
len(grounds), len(preds_l)

(1101, 1101)

In [122]:
preds_l

[['Fear', 'Surprise'],
 ['Fear', 'Surprise'],
 ['Fear', 'Sadness'],
 ['Joy'],
 ['Joy'],
 ['Neutral'],
 ['Fear', 'Surprise'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Fear', 'Surprise'],
 ['Neutral'],
 ['Fear'],
 ['Sadness'],
 ['Sadness'],
 ['Sadness'],
 ['Anger', 'Fear'],
 ['Sadness', 'Surprise'],
 ['Surprise'],
 ['Joy'],
 ['Sadness'],
 ['Anger', 'Fear'],
 ['Anger'],
 ['Anger', 'Fear'],
 ['Anger', 'Fear'],
 ['Anger', 'Surprise'],
 ['Anger', 'Fear'],
 ['Surprise', 'Joy'],
 ['Anger', 'Sadness'],
 ['Anger', 'Fear'],
 ['Fear'],
 ['Neutral'],
 ['Fear', 'Surprise'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Fear'],
 ['Neutral'],
 ['Fear'],
 ['Neutral'],
 ['Neutral'],
 ['Fear', 'Sadness'],
 ['Neutral'],
 ['Anger', 'Disgust'],
 ['Sadness'],
 ['Fear', 'Joy'],
 ['Anger', 'Fear'],
 ['Fear'],
 ['Fear', 'Surprise'],
 ['Anger', 'Surprise'],
 ['Anger', 'Fear'],
 ['Fear'],
 ['Fear', 'Sadness'],
 ['Anger', 'Surprise'],
 ['Fear', 'Sadness'],
 ['Fear', 'Sadness'],
 ['Anger', 'Fear'],

In [123]:
mlb = MultiLabelBinarizer()

In [124]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(preds_l)

/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['Concern'] will be ignored
  warnings.warn(


In [125]:
y_pred_mhot.shape

(1101, 7)

In [126]:
y_pred_mhot.shape

(1101, 7)

In [127]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=mlb.classes_, digits=3))

              precision    recall  f1-score   support

       Anger      0.545     0.574     0.559       399
     Disgust      0.250     0.213     0.230        47
        Fear      0.405     0.545     0.465       235
         Joy      0.593     0.430     0.499       244
     Neutral      0.321     0.286     0.303        63
     Sadness      0.517     0.519     0.518       314
    Surprise      0.650     0.480     0.553       279

   micro avg      0.514     0.498     0.506      1581
   macro avg      0.469     0.435     0.447      1581
weighted avg      0.527     0.498     0.507      1581
 samples avg      0.528     0.511     0.496      1581



/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
